In [1]:
## Carga de librerías
import cupy as cp
import cudf
import matplotlib.pyplot as plt
import numpy as np
import time
cp.random.seed(0)

In [2]:
## Funciones auxiliares
def change_dates(df):
    #changes fecha_inicio_sintomas according to new criteria
    filt_df2 = (df.fecha_inicio_sintomas.isnull()) #filter fecha_inicio_sintomas = inexistant
    n_size = (df[filt_df2]).shape[0]
    df["fecha_inicio_sintomas"] = df["fecha_inicio_sintomas"].fillna(0)
    df["fecha_inicio_sintomas"] = df["fecha_inicio_sintomas"].astype('int')
    df["fecha_apertura"] = df["fecha_apertura"].astype('int')
    b = cp.array(df["fecha_apertura"].values)
    a = cp.array(df["fecha_inicio_sintomas"].values)
    a[filt_df2.values] = b[filt_df2.values] - cp.random.randint(0,9,a[filt_df2.values].shape)
    filt_a = cp.where(a<0)
    a[filt_a] = 0
    df = df.drop(["fecha_inicio_sintomas"],axis=1)
    df["fecha_inicio_sintomas"] = a
    return df

In [3]:
tiempo1 = []
tiempo2 = []
tiempo3 = []
N = 10
for cont in range(N):
    print(cont)
    start1 = time.time()
    start2 = time.time()
    ## Carga de datos, sacamos las series temporal
    direccion = '12en20.csv'
    columnas = ['residencia_provincia_nombre','residencia_departamento_nombre','fecha_inicio_sintomas',
                'fecha_apertura','clasificacion_resumen']
    data = cudf.read_csv(direccion,sep=",",quotechar='"', usecols=columnas,
                       parse_dates=["fecha_inicio_sintomas","fecha_apertura"],na_values=['']) 
    end2 = time.time()
    data = data[data["clasificacion_resumen"] == "Confirmado"] #filter confirmed cases
    df = cudf.DataFrame(data) 
    df = df.drop(['clasificacion_resumen'], axis=1) #drop clasificacion_resumen bc previous filter of confirmed cases
    filt_df1 = (df.fecha_inicio_sintomas.notnull()) #filter fecha_inicio_sintomas = inexistant
    inicio_epidemia = (df.loc[filt_df1,"fecha_inicio_sintomas"]).min() #first symptoms of a person registered
    ultima_actualizacion_sintomas = (df.loc[filt_df1,"fecha_inicio_sintomas"]).max() #last day symptoms of a person registered
    ultima_actualizacion_apertura = (df["fecha_apertura"]).max()
    df["fecha_inicio_sintomas"] -= inicio_epidemia #correcting by inicio_epidemia 
    df["fecha_apertura"] -= inicio_epidemia #correcting by inicio_epidemia
    df.fecha_inicio_sintomas = df.fecha_inicio_sintomas.dt.days #change to int, ditch days 
    df.fecha_apertura = df.fecha_apertura.dt.days #change to int, ditch days
    #print("primer sintoma de persona confirmada: ",inicio_epidemia)
    #print("ultimo sintoma de persona confirmada: ",ultima_actualizacion_sintomas)
    #print("ultima apertura de persona confirmada: ",ultima_actualizacion_apertura)
    df = change_dates(df) # we have replaced all bad or undefined dates
    df = df[["residencia_provincia_nombre","fecha_inicio_sintomas"]]
    df2 = df.groupby(["residencia_provincia_nombre","fecha_inicio_sintomas"]).size().reset_index() 
    df2 = df2.rename(columns={0:'casos'})
    df3 = df2.sort_values(["residencia_provincia_nombre","fecha_inicio_sintomas"],ascending=[True,True])
    #let's list all the provinces that have cases
    #we'll get those times series then, with cudf grouping
    provincias = cudf.Series(df3["residencia_provincia_nombre"].unique())
    # there are 24 provinces (23 + CABA) + SIN ESPECIFICAR cases
    rows = provincias.shape[0]
    cols = df2["fecha_inicio_sintomas"].max()
    time_series = cp.zeros(shape=(rows,cols+1)) #time series of new cases / day
    time_series_ac = cp.zeros(shape=(rows,cols+1)) # time series of attack rate / day (attack rate = 14 days cumulate of new cases)
    j = 0
    for i in provincias.to_pandas():
      df_tmp = df3[df3["residencia_provincia_nombre"]==i]
      tmp_dates = cp.array(df_tmp["fecha_inicio_sintomas"].values)
      tmp_cases = cp.array(df_tmp["casos"].values)
      time_series[j,tmp_dates]= tmp_cases
      j = j + 1
    for i in range(rows):
      for j in range(13,cols):
        time_series_ac[i,j] = cp.sum(time_series[i,j-13:j+1])
    # Ahora, lo hacemos en serio, tipo todas las provincias contra todas las provincias, y guardamos los resultados
    start3 = time.time()
    localidades_lagmax = cp.zeros(shape=(rows,rows))
    localidades_corrmax = cp.zeros(shape=(rows,rows))
    cpts = cols
    lags = cp.arange(-cpts + 1, cpts)
    i = 0
    while i < rows:
        j = 0
        prov_tmp_1 = time_series_ac[i,:-10]
        while j < rows:
            prov_tmp_2 = time_series_ac[j,:-10]
            ccov = cp.correlate(prov_tmp_1  - prov_tmp_1 .mean(), prov_tmp_2 - prov_tmp_2.mean(), mode='full')
            ccor = ccov / (cpts * prov_tmp_1 .std() * prov_tmp_2.std())
            localidades_lagmax[i,j] = lags[cp.argmax(ccor)]
            localidades_corrmax[i,j] = cp.max(ccor)
            j = j + 1
        i = i +1
    end3 = time.time()
    end1 = time.time()
    tiempo1.append(end1 - start1)
    tiempo2.append(end2 - start2)
    tiempo3.append(end3 - start3)

0


/home/denise/miniconda3/envs/gpup/lib/python3.7/site-packages/cudf/core/frame.py:2627: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


1
2
3
4
5
6
7
8
9


In [4]:
print('Tiempo 1: ',np.array(tiempo1).mean(),np.array(tiempo1).std())
print('Tiempo 2: ',np.array(tiempo2).mean(),np.array(tiempo2).std())
print('Tiempo 3: ',np.array(tiempo3).mean(),np.array(tiempo3).std())

Tiempo 1:  7.965578770637512 2.7952278033051456
Tiempo 2:  1.93735511302948 2.085674276767018
Tiempo 3:  3.424283504486084 0.5658063328757428
